# Generic analysis with dynamic indicators and stability time

### Execute this cell for better plots

In [2]:
%matplotlib widget

### Libraries

In [32]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import os
from tqdm.autonotebook import tqdm
from numba import njit, prange

DPI = 300

## Directories (local computer)

In [33]:
outdir = "../data"
inputdir = "../data"
imgdir = "../img"

## Directories (SWAN instance + CERNBox file system)

In [34]:
outdir = "../data"
inputdir = "../data"
imgdir = "../img"

## Data Filename list

In [35]:
init_file = "henon_4d_init_eps_0_0_mu_0_0_id_basic_view.hdf5"

displacement_file = "henon_4d_displacement_eps_0_0_mu_0_0_id_basic_view_subid_1e-14.hdf5"
inversion_file = "henon_4d_inverse_tracking_eps_0_0_mu_0_0_id_basic_view_subid_no_kick.hdf5"
inversion_gauss_file = "henon_4d_inverse_tracking_eps_0_0_mu_0_0_id_basic_view_subid_gauss_kick.hdf5"
inversion_uniform_file = "henon_4d_inverse_tracking_eps_0_0_mu_0_0_id_basic_view_subid_unif_kick.hdf5"
inversion_gauss_forward_file = "henon_4d_inverse_tracking_eps_0_0_mu_0_0_id_basic_view_subid_gauss_kick_forward.hdf5"
inversion_uniform_forward_file = "henon_4d_inverse_tracking_eps_0_0_mu_0_0_id_basic_view_subid_unif_kick_forward.hdf5"
tracking_file = "henon_4d_long_track_eps_0_0_mu_0_0_id_basic_view.hdf5"
orto_displacement_file = "henon_4d_orto_displacement_eps_0_0_mu_0_0_id_basic_view_subid_1e-14.hdf5"
full_track_file = "henon_4d_verbose_track_eps_0_0_mu_0_0_id_basic_view.hdf5"

## Load Initial conditions

In [36]:
initial_conditions = h5py.File(os.path.join(inputdir, init_file), mode="r")

## Long tracking

In [37]:
long_tracking = h5py.File(os.path.join(inputdir, tracking_file), mode="r")

In [38]:
plt.figure()
plt.imshow(np.log10(long_tracking["stability_time"]), origin="lower", extent=[0,1,0,1])

plt.xlabel("$X_0$")
plt.ylabel("$Y_0$")
plt.colorbar(label="Stability time $(\\log10)$")
plt.title("Stability Time Heatmap")

plt.savefig(os.path.join(imgdir, "henon_4d_stab_time.png"), dpi=DPI)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [39]:
plt.close("all")

## Lyapunov

In [40]:
displacement = h5py.File(os.path.join(inputdir, displacement_file), mode='r')

In [41]:
turn_samples = np.logspace(np.log10(displacement.attrs["min_turns"]), np.log10(displacement.attrs["max_turns"]), displacement.attrs["samples"], dtype=int)

### Basic fast Lyapunov indicator

In [42]:
LI = []
for t in turn_samples:
    sample = displacement[str(t)]
    LI.append(np.log10(np.sqrt(
        np.power(sample["x"][0] - sample["x"][1], 2) +
        np.power(sample["px"][0] - sample["px"][1], 2) +
        np.power(sample["y"][0] - sample["y"][1], 2) +
        np.power(sample["py"][0] - sample["py"][1], 2)
    ) / displacement.attrs["displacement"] ) / t)
LI = np.asarray(LI)

In [53]:
plt.figure()

plt.imshow(LI[-1], origin="lower", extent=[0,1,0,1])
plt.colorbar(label="Indicator value")
plt.title("Fast Lyapunov Indicator $(t={:.2e})$".format(turn_samples[-1]))

plt.xlabel("$X_0$")
plt.ylabel("$Y_0$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$Y_0$')

In [62]:
plt.figure()
for i, t in enumerate(turn_samples):
    plt.imshow(LI[i], origin="lower", extent=[0,1,0,1])
    plt.colorbar(label="Indicator value")
    plt.title("Fast Lyapunov Indicator $(t={:.2e})$".format(t))

    plt.xlabel("$X_0$")
    plt.ylabel("$Y_0$")
    
    plt.savefig(os.path.join(imgdir, "henon_4d_LI_{}.png".format(i)), dpi=DPI)
    plt.clf() if i != len(turn_samples) - 1 else plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Invariant Lyapunov (LEI)

In [55]:
orto_displacement = h5py.File(os.path.join(inputdir, orto_displacement_file), mode='r')

In [56]:
turn_samples = np.logspace(np.log10(orto_displacement.attrs["min_turns"]), np.log10(orto_displacement.attrs["max_turns"]), orto_displacement.attrs["samples"], dtype=int)

In [57]:
sample = orto_displacement[str(turn_samples[-1])]

In [52]:
LEI_1 = []
for t in turn_samples: 
    sample = orto_displacement[str(t)]

    t11 = sample["x"][1] - sample["x"][0]
    t12 = sample["px"][1] - sample["px"][0]
    t13 = sample["y"][1] - sample["y"][0]
    t14 = sample["py"][1] - sample["py"][0]

    t21 = sample["x"][2] - sample["x"][0]
    t22 = sample["px"][2] - sample["px"][0]
    t23 = sample["y"][2] - sample["y"][0]
    t24 = sample["py"][2] - sample["py"][0]

    t31 = sample["x"][3] - sample["x"][0]
    t32 = sample["px"][3] - sample["px"][0]
    t33 = sample["y"][3] - sample["y"][0]
    t34 = sample["py"][3] - sample["py"][0]

    t41 = sample["x"][4] - sample["x"][0]
    t42 = sample["px"][4] - sample["px"][0]
    t43 = sample["y"][4] - sample["y"][0]
    t44 = sample["py"][4] - sample["py"][0]

    tm = np.transpose(np.array([
            [t11, t12, t13, t14],
            [t21, t22, t23, t24],
            [t31, t32, t33, t34],
            [t41, t42, t43, t44]
        ]), axes=(2, 3, 0, 1))
    tmt = np.transpose(tm, axes=(0, 1, 3, 2))
    
    LEI_1.append(np.log10(np.sqrt(np.trace(np.matmul(tmt,tm), axis1=2, axis2=3))/orto_displacement.attrs["displacement"])/t)
LEI_1 = np.asarray(LEI_1)

In [59]:
plt.figure()

plt.imshow(LEI_1[-1], origin="lower", extent=[0,1,0,1])
plt.colorbar(label="Indicator value")
plt.title("Invariant Lyapunov Indicator $(t={:.2e})$".format(turn_samples[-1]))

plt.xlabel("$X_0$")
plt.ylabel("$Y_0$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$Y_0$')

In [61]:
plt.figure()
for i, t in enumerate(turn_samples):
    plt.imshow(LEI_1[i], origin="lower", extent=[0,1,0,1])
    plt.colorbar(label="Indicator value")
    plt.title("Invariant Lyapunov Indicator $(t={:.2e})$".format(t))

    plt.xlabel("$X_0$")
    plt.ylabel("$Y_0$")
    
    plt.savefig(os.path.join(imgdir, "henon_4d_LEI_{}.png".format(i)), dpi=DPI)
    plt.clf() if i != len(turn_samples) - 1 else plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
plt.figure()
for i, t in tqdm(enumerate(turn_samples), total=len(turn_samples)):
    plt.scatter(
        LI[i].flatten(),
        LEI_1[i].flatten(),
        s=1, label="data",
        c=np.log10(long_tracking["stability_time"]).flatten(), cmap="viridis"
    )
    plt.plot(
        np.linspace(np.nanmin(LI[20]), np.nanmax(LI[20])),
        np.linspace(np.nanmin(LI[20]), np.nanmax(LI[20])),
        c="red", label="$x=y$", linewidth=1)
    plt.colorbar(label="Stability time $(\\log10)$")
    plt.xlabel("Fast Lyapunov")
    plt.ylabel("Invariant Lyapunov")
    plt.title("Direct comparison between FastLI and LEI $(t={:.2e})$".format(t))
    
    plt.savefig(os.path.join(imgdir, "henon_4d_vs_LI_LEI_{}.png".format(i)), dpi=DPI)
    plt.clf() if i != len(turn_samples) - 1 else plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

  0%|          | 0/101 [00:00<?, ?it/s]

## Frequency/Tune Analysis

In [67]:
@njit(parallel=True)
def select_indices(l, c, r, data):
    p1 = np.empty(len(l), dtype=numba.int32)
    p2 = np.empty(len(l), dtype=numba.int32)
    vp1 = np.empty(len(l))
    vp2 = np.empty(len(l))
    for i in prange(data.shape[1]):
        if data[l[i], i] > data[r[i], i]:
            p1[i] = l[i]
            p2[i] = c[i]
            vp1[i] = data[p1[i], i]
            vp2[i] = data[p2[i], i]
        else:
            p1[i] = c[i]
            p2[i] = r[i]
            vp1[i] = data[p1[i], i]
            vp2[i] = data[p2[i], i]
    return p1, p2, vp1, vp2

@njit()
def interpolation(data, index):
    if np.any(np.isnan(data)):
        value = np.nan
    elif index == 0:
        value = 1
    else:
        if index == len(data) - 1:
            index -= 1
        cf1 = np.absolute(data[index - 1])
        cf2 = np.absolute(data[index])
        cf3 = np.absolute(data[index + 1])
        if cf3 > cf1:
            p1 = cf2
            p2 = cf3
            nn = index
        else:
            p1 = cf1
            p2 = cf2
            nn = index - 1            
        p3 = np.cos(2 * np.pi / len(data))
        value = (
            (nn / len(data)) + (1/np.pi) * np.arcsin(
                np.sin(2*np.pi/len(data)) * 
                ((-(p1+p2*p3)*(p1-p2) + p2*np.sqrt(p3**2*(p1+p2)**2 - 2*p1*p2*(2*p3**2-p3-1)))/(p1**2 + p2**2 + 2*p1*p2*p3))
            )
        )
    return np.absolute(1 - value)

In [68]:
full_track = h5py.File(os.path.join(inputdir, full_track_file), mode='r')

In [69]:
tune_x = np.empty((500,500))
tune_y = np.empty((500,500))
tune_x1 = np.empty((500,500))
tune_y1 = np.empty((500,500))
tune_x2 = np.empty((500,500))
tune_y2 = np.empty((500,500))

In [70]:
for i in tqdm(range(500)):
    data_x = full_track["coords/x"][:,i]
    data_px = full_track["coords/px"][:,i]
    data_y = full_track["coords/y"][:,i]
    data_py = full_track["coords/py"][:,i]
    for k in range(500):
        signal = data_x[:, k] + 1j * data_px[:, k]
        fft = np.absolute(np.fft.fft(signal * np.hanning(signal.shape[0])))
        value = np.argmax(fft)
        value = interpolation(fft, value)
        tune_x[i, k] = value
        
        signal = data_y[:, k] + 1j * data_py[:, k]
        fft = np.absolute(np.fft.fft(signal * np.hanning(signal.shape[0])))
        value = np.argmax(fft)
        value = interpolation(fft, value)
        tune_y[i, k] = value
        
        signal = data_x[:len(data_x)//2, k] + 1j * data_px[:len(data_x)//2, k]
        fft = np.absolute(np.fft.fft(signal * np.hanning(signal.shape[0])))
        value = np.argmax(fft)
        value = interpolation(fft, value)
        tune_x1[i, k] = value
        
        signal = data_y[:len(data_x)//2, k] + 1j * data_py[:len(data_x)//2, k]
        fft = np.absolute(np.fft.fft(signal * np.hanning(signal.shape[0])))
        value = np.argmax(fft)
        value = interpolation(fft, value)
        tune_y1[i, k] = value
        
        signal = data_x[len(data_x)//2:, k] + 1j * data_px[len(data_x)//2:, k]
        fft = np.absolute(np.fft.fft(signal * np.hanning(signal.shape[0])))
        value = np.argmax(fft)
        value = interpolation(fft, value)
        tune_x2[i, k] = value
        
        signal = data_y[len(data_x)//2:, k] + 1j * data_py[len(data_x)//2:, k]
        fft = np.absolute(np.fft.fft(signal * np.hanning(signal.shape[0])))
        value = np.argmax(fft)
        value = interpolation(fft, value)
        tune_y2[i, k] = value

  0%|          | 0/500 [00:00<?, ?it/s]

In [72]:
plt.figure()

plt.imshow(tune_x, origin="lower", extent=[0,1,0,1])
plt.colorbar(label="Tune $X$ [$2\\pi$ units]")
plt.xlabel("$X_0$")
plt.ylabel("$Y_0$")
plt.title("Tune $X$ heatmap")

plt.savefig(os.path.join(imgdir, "henon_4d_tunex.png"), dpi=DPI)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [73]:
plt.figure()

plt.imshow(tune_y, origin="lower", extent=[0,1,0,1])
plt.colorbar(label="Tune $X$ [$2\\pi$ units]")
plt.xlabel("$X_0$")
plt.ylabel("$Y_0$")
plt.title("Tune $Y$ heatmap")

plt.savefig(os.path.join(imgdir, "henon_4d_tuney.png"), dpi=DPI)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [74]:
plt.figure()

known_x_tune = 0.168
known_y_tune = 0.201

min_resonance = 2
max_resonance = 6
colors = ["red", "blue", "green", "orange", "cyan"]
alpha = 1.0

plt.axhline(known_y_tune, color="grey", label="known\imgdirp freq.")
plt.axvline(known_x_tune, color="grey")

x = np.linspace(0,1,100)

for i in list(range(min_resonance, max_resonance + 1))[::-1]:
    for j in range(1, i):
        nx = j
        ny = i - j
        for q in range(0, i+1):
            plt.plot(x, q/ny - nx / ny * x,
                     color=colors[i-min_resonance], alpha=alpha, zorder=1)
            plt.plot(x, q/(-ny) - nx / (-ny) * x,
                     color=colors[i-min_resonance], alpha=alpha, zorder=1)
            plt.plot(x, q/ny - (-nx) / ny * x,
                     color=colors[i-min_resonance], alpha=alpha, zorder=1)
            plt.plot(x, q/(-ny) - (-nx) / (-ny) * x,
                     color=colors[i-min_resonance], alpha=alpha, zorder=1)
    for j in range(i+1):
        plt.axvline(j/i, color=colors[i-min_resonance], alpha=alpha, zorder=1) if j != 0 else plt.axvline(
            j/i, color=colors[i-min_resonance], label="res {}".format(i), alpha=alpha, zorder=1)
        plt.axhline(j/i, color=colors[i-min_resonance], alpha=alpha, zorder=1)

plt.scatter(tune_x.flatten(), tune_y.flatten(), s=0.5,
            marker="x", c=np.log10(long_tracking["stability_time"]), cmap='viridis', label="data", zorder=1)
plt.colorbar(label="Stability time (log10)")
        
plt.xlim(0,1)
plt.ylim(0,1)

plt.xlabel("tune $x$ [$2\\pi$ units]")
plt.ylabel("tune $y$ [$2\\pi$ units]")
plt.legend(ncol=2)
plt.title("Resonance Plot")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Resonance Plot')

In [31]:
plt.figure()

plt.imshow(
    np.log10(np.sqrt(np.power(tune_x1 - tune_x2, 2) + np.power(tune_y1 - tune_y2, 2))),
    origin="lower"
)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
plt.figure()
plt.scatter(LEI_1[30],  np.log10(np.sqrt(np.power(tune_x1 - tune_x2, 2) + np.power(tune_y1 - tune_y2, 2))), s=1, c=np.log10(long_tracking["stability_time"]).flatten(), cmap="viridis")
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()

plt.imshow(
    (np.sqrt(np.power(tune_x1 - tune_x2, 2) + np.power(tune_y1 - tune_y2, 2))),
    origin="lower"
)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
inversion_pure = h5py.File(os.path.join(inputdir, inversion_file), mode='r')
inversion_gauss = h5py.File(os.path.join(inputdir, inversion_gauss_file), mode='r')
inversion_uniform = h5py.File(os.path.join(inputdir, inversion_uniform_file), mode='r')
inversion_gauss_forward = h5py.File(os.path.join(inputdir, inversion_gauss_forward_file), mode='r')
inversion_uniform_forward = h5py.File(os.path.join(inputdir, inversion_uniform_forward_file), mode='r')

In [24]:
turn_sampling = np.logspace(np.log10(inversion_pure.attrs["min_turns"]), np.log10(inversion_pure.attrs["max_turns"]), inversion_pure.attrs["samples"], dtype=np.int)

In [26]:
RE = []
RE_U = []
RE_G = []
RE_U_F = []
RE_G_F = []

for t in tqdm(turn_sampling):
    RE.append(np.sqrt(
        + np.power(initial_conditions["coords/x"][...] - inversion_pure[str(t)]["x"][...], 2)
        + np.power(initial_conditions["coords/px"][...] - inversion_pure[str(t)]["px"][...], 2)
        + np.power(initial_conditions["coords/y"][...] - inversion_pure[str(t)]["y"][...], 2)
        + np.power(initial_conditions["coords/py"][...] - inversion_pure[str(t)]["py"][...], 2)
    ))
    RE_U.append(np.sqrt(
        + np.power(initial_conditions["coords/x"][...] - inversion_uniform[str(t)]["x"][...], 2)
        + np.power(initial_conditions["coords/px"][...] - inversion_uniform[str(t)]["px"][...], 2)
        + np.power(initial_conditions["coords/y"][...] - inversion_uniform[str(t)]["y"][...], 2)
        + np.power(initial_conditions["coords/py"][...] - inversion_uniform[str(t)]["py"][...], 2)
    ))
    RE_G.append(np.sqrt(
        + np.power(initial_conditions["coords/x"][...] - inversion_gauss[str(t)]["x"], 2)
        + np.power(initial_conditions["coords/px"][...] - inversion_gauss[str(t)]["px"], 2)
        + np.power(initial_conditions["coords/y"][...] - inversion_gauss[str(t)]["y"], 2)
        + np.power(initial_conditions["coords/py"][...] - inversion_gauss[str(t)]["py"], 2)
    ))
    RE_U_F.append(np.sqrt(
        + np.power(initial_conditions["coords/x"][...] - inversion_uniform_forward[str(t)]["x"][...], 2)
        + np.power(initial_conditions["coords/px"][...] - inversion_uniform_forward[str(t)]["px"][...], 2)
        + np.power(initial_conditions["coords/y"][...] - inversion_uniform_forward[str(t)]["y"][...], 2)
        + np.power(initial_conditions["coords/py"][...] - inversion_uniform_forward[str(t)]["py"][...], 2)
    ))
    RE_G_F.append(np.sqrt(
        + np.power(initial_conditions["coords/x"][...] - inversion_gauss_forward[str(t)]["x"][...], 2)
        + np.power(initial_conditions["coords/px"][...] - inversion_gauss_forward[str(t)]["px"][...], 2)
        + np.power(initial_conditions["coords/y"][...] - inversion_gauss_forward[str(t)]["y"][...], 2)
        + np.power(initial_conditions["coords/py"][...] - inversion_gauss_forward[str(t)]["py"][...], 2)
    ))

RE = np.asarray(RE)
RE_U = np.asarray(RE_U)
RE_G = np.asarray(RE_G)
RE_U_F = np.asarray(RE_U_F)
RE_G_F = np.asarray(RE_G_F)

  0%|          | 0/61 [00:00<?, ?it/s]

In [29]:
plt.figure()
plt.imshow(np.log10(RE[-1]), origin="lower")
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [49]:
plt.figure()
plt.scatter(np.log10(RE[-1]), LI[-20], s=1, c=np.log10(long_tracking["stability_time"]), cmap="viridis")
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [54]:
plt.figure()
plt.scatter(np.log10(RE[-20]), np.log10(RE_U[-20]), s=1, c=np.log10(long_tracking["stability_time"]), cmap="viridis")
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
plt.figure()
plt.scatter(np.log10(RE_G[20]), np.log10(RE_U[20]), s=1, c=np.log10(long_tracking["stability_time"]), cmap="viridis")
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …